In [32]:
import pyspark
from pyspark.pandas import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col, array_contains

spark = SparkSession.builder.appName('MusicGen').getOrCreate()

In [5]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 1.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 55.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.0 MB/s eta 0:00:00ta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 28.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 60.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 67.8 MB/s eta 0:00:00
     ━━━━━━━

In [61]:
from keras.layers import (
    Input, Dense, Activation, TimeDistributed, Softmax, TextVectorization, Reshape,
    RepeatVector, Conv1D, Bidirectional, AveragePooling1D, UpSampling1D, Embedding,
    Concatenate, GlobalAveragePooling1D, LSTM, Multiply, MultiHeadAttention
)
from keras.models import Model
import tensorflow as tf
import keras
import numpy as np

In [62]:
schema = StructType() \
      .add("title", StringType(),True) \
      .add("tag", StringType(), True) \
      .add("artist", StringType(), True) \
      .add("year", IntegerType(), True) \
      .add("views", IntegerType(), True) \
      .add("features", StringType(), True) \
      .add("lyrics", StringType(), True) \
      .add("id", IntegerType(), True) \
      .add("language_cld3", StringType(), True) \
      .add("language_ft", StringType(), True) \
      .add("language", StringType(), True)

# df = spark.read.csv("song_lyrics.csv")
# df.printSchema()
df = spark.read.format("csv") \
      .option("header", True) \
      .option("multiLine", True) \
      .option("escape","\"") \
      .schema(schema) \
      .load("song_lyrics.csv")
df.printSchema()

root
 |-- title: string (nullable = true)
 |-- tag: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- views: integer (nullable = true)
 |-- features: string (nullable = true)
 |-- lyrics: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- language_cld3: string (nullable = true)
 |-- language_ft: string (nullable = true)
 |-- language: string (nullable = true)



In [65]:
cols = ("artist", "year", "views", "id", "language_cld3", "language_ft")

df = df.drop(*cols)

In [70]:
from pyspark.sql.functions import monotonically_increasing_id 

df = df.select("*").withColumn("id", monotonically_increasing_id())

In [71]:
df.printSchema()

root
 |-- title: string (nullable = true)
 |-- tag: string (nullable = true)
 |-- features: string (nullable = true)
 |-- lyrics: string (nullable = true)
 |-- language: string (nullable = true)
 |-- id: long (nullable = false)



In [89]:
indexes = {1, 2, 3, 4}
# df.where(col("id").isin(indexes)).show(4)
df.filter(col("id").isin(indexes)).show(1)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [19]:
wanted_tag = 'pop'
df_ = DataFrame(df.filter(df.tag == wanted_tag))
n_rows = df_.count()

In [20]:
n_rows

title            2138552
tag              2138587
artist           2138587
year             2138587
views            2138587
features         2138587
lyrics           2138587
id               2138587
language_cld3    2094640
language_ft      2077183
language         2030273
dtype: int64

In [24]:
df_.iloc[0]["lyrics"]

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [22]:
class BatchDataset(tf.keras.utils.Sequence):    
    def __init__(self, dataset_spark, batch_size, dataset_len):
        self.batch_size = batch_size
        self.dataset_spark = dataset_spark
        self.dataset_full_len = dataset_len
    
    def __len__(self):
        return np.ceil(self.dataset_full_len / self.batch_size)

    def __getitem__(self, idx):
        # TODO: Rever uma forma performatica para fazer isso.
        rows = self.dataset_spark.iloc[idx * self.batch_size:(idx+1) * self.batch_size]
        X = rows["lyrics"]
        return X
        
    
#     def _shuffle(self):
#         import random
#         temp = list(zip(self.list_of_files, self.list_of_labels))
#         random.shuffle(temp)
#         res1, res2 = zip(*temp)
#         # res1 and res2 come out as tuples, and so must be converted to lists.
#         self.list_of_files, self.list_of_labels = list(res1), list(res2)

#     def on_epoch_end(self):
#         self._shuffle()

vocab_size=10_000
n_grams=10
vectorize_layer = TextVectorization(
        max_tokens=vocab_size, output_sequence_length=n_grams
    )
dataset = BatchDataset(df_, 20_000, n_rows)

In [23]:
vectorize_layer.adapt(dataset)

TypeError: Could not build a TypeSpec for 0      [Chorus]\nWhat are words worth?\nWhat are word...
1      [Verse 1]\nIn December, drinking horchata\nI'd...
2      [Chorus]\nIn the night, I hear 'em talk\nThe c...
3      [Intro: Connie Mitchell]\nFlashing lights (Lig...
4      [Produced by The-Dream and Tricky Stewart]\n\n...
5      [Verse 1]\nI'm more than just an option (Hey, ...
6      [Verse 1]\nI feel like I wanna smack somebody\...
7      [Chorus]\nI see you drivin' 'round town with t...
8      [Verse 1]\nNow when the sun come up\nI'll be t...
9      [Verse 1: Kanye West]\nI got a big ego (Ha ha ...
10     [Pre-Chorus: Bruno Mars]\nI wanna be a billion...
11     [Intro]\nMum mum mum mah\nMum mum mum mah\nMum...
12     [Intro: Raekwon]\nVisual visual, JB\nYo, Ye, w...
13     [Verse 1]\nWho's in the bunker? Who's in the b...
14     [Intro]\nI know that we are young, and I know ...
15     [Bridge: Vika Jigulina]\nWhen you gonna stop b...
16     Voila, la la la c'est la\nVoila, la la 

In [ ]:
def predict_word(seq_len, latent_dim, vocab_size):
    input_layer = Input(shape=(seq_len-1,))
    x = input_layer
    x = Embedding(vocab_size, latent_dim, name='embedding', mask_zero=True)(x)
    x = MultiHeadAttention(num_heads=3, key_dim=2)(x, value=x)
    x = GlobalAveragePooling1D()(x)
    latent_rep = x
    x = Dense(vocab_size)(x)
    x = Softmax()(x)
    return Model(input_layer, x), Model(input_layer, latent_rep)

predictor, latent = predict_word(n_grams, 15, vocab_size)
predictor.summary()
#opt = keras.optimizers.SGD(learning_rate=1, momentum=0.9)
opt = keras.optimizers.Nadam(learning_rate=0.1)
loss_fn = keras.losses.SparseCategoricalCrossentropy(
    ignore_class=1,
    name="sparse_categorical_crossentropy",
)

predictor.compile(loss=loss_fn, optimizer=opt, metrics=["accuracy"])

In [ ]:
def get_last_token(x):
        """
        Function to map the dataset to (x, y) pairs.
        The y is last token of x.
        x is output of vectorization - last token.
        """
        x_ = self.vectorize_layer(x)
        y_ = x_[:,-1:]
        x_ = x_[:, :-1]
        return x_, y_

history = predictor.fit(dataset.map(separar_ultimo_token), epochs=40, verbose=1)